<a href="https://colab.research.google.com/github/Rizil-Patel/FeatureFusionML/blob/main/ImageProcessingProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_excel('Train9.xlsx')

df.to_csv('Train9.csv', index=True)

In [ ]:
import pandas as pd
train_df = pd.read_csv('Train9.csv')
train_df.columns.values[0] = 'index'
train_df.head()

,index,image_link,group_id,entity_name,entity_value
0,0,https://m.media-amazon.com/images/I/619I+wNtdL...,564709,item_weight,75 kilogram
1,1,https://m.media-amazon.com/images/I/61lV+HEkCv...,993359,item_weight,50 gram
2,2,https://m.media-amazon.com/images/I/71h79xVquL...,993359,item_weight,100 gram
3,3,https://m.media-amazon.com/images/I/715AjsGFYs...,993359,item_weight,100 gram
4,4,https://m.media-amazon.com/images/I/71wYWr7qtZ...,993359,item_weight,100 gram


In [ ]:
import os
from pathlib import Path

def process_images_and_update_csv(df, image_column, download_folder, output_csv):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    image_links = df[image_column].dropna().unique()

    utils.download_images(image_links, download_folder, allow_multiprocessing=True)

    df['image_path'] = df[image_column].apply(
        lambda link: os.path.join(download_folder, Path(link).name) if pd.notna(link) else None
    )

    df.to_csv(output_csv, index=False)

In [ ]:
import os
import pandas as pd
from pathlib import Path
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

download_folder = '/content/downloaded_images'

if not os.path.exists(download_folder):
    os.makedirs(download_folder)

input_csv = 'Train9.csv'
df = pd.read_csv(input_csv)

image_column = 'image_link'
df = df.head(5000)

def download_image(image_link, download_folder):
    try:
        if pd.isna(image_link):
            return None, True

        image_name = Path(image_link).name
        image_path = os.path.join(download_folder, image_name)

        response = requests.get(image_link, stream=True)
        response.raise_for_status()

        with open(image_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        return image_path, True
    except Exception as e:
        print(f"Error downloading {image_link}: {e}")
        return None, False

def download_images_from_links(df, image_column, download_folder, num_threads=8):
    image_links = df[image_column]

    image_paths = []

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = {executor.submit(download_image, link, download_folder): link for link in image_links}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading images"):
            link = futures[future]
            try:
                image_path, success = future.result()
                image_paths.append(image_path)
            except Exception as e:
                print(f"Exception occurred for {link}: {e}")
                image_paths.append(None)

    return image_paths

df['image_path'] = download_images_from_links(df, image_column, download_folder)

output_csv = 'Train9_image.csv'
df.to_csv(output_csv, index=False)

print(f"Image download complete. Data saved to {output_csv}")


Error downloading https://m.media-amazon.com/images/I/J2DXsUjR8ay.jpg: 404 Client Error: Not Found for url: https://m.media-amazon.com/images/I/J2DXsUjR8ay.jpg


Error downloading https://m.media-amazon.com/images/I/mWyQ79S76i.jpg: 400 Client Error: Bad Request for url: https://m.media-amazon.com/images/I/mWyQ79S76i.jpg


Image download complete. Data saved to Train9_image.csv


In [ ]:
import cv2
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

output_csv = 'Train9_image.csv'
df = pd.read_csv(output_csv)

resize_width = 224
resize_height = 224

def preprocess_image(image_path):
    """ Preprocess the image by resizing, converting to grayscale, and normalizing. """
    try:
        image = cv2.imread(image_path)

        if image is None:
            print(f"Image not found at {image_path}.")
            return None

        image_resized = cv2.resize(image, (resize_width, resize_height))

        image_gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)

        image_normalized = image_gray / 255.0

        return image_normalized
    except Exception as e:
        print(f"Error preprocessing image {image_path}: {e}")
        return None

preprocessed_images = []
for image_path in tqdm(df['image_path'], desc="Preprocessing images"):
    preprocessed_image = preprocess_image(image_path)
    preprocessed_images.append(preprocessed_image)

preprocessed_folder = '/content/preprocessed_images'
if not os.path.exists(preprocessed_folder):
    os.makedirs(preprocessed_folder)

for i, img in enumerate(preprocessed_images):
    if img is not None:
        np.save(os.path.join(preprocessed_folder, f'preprocessed_images_{i}.npy'), img)

print(f"Image preprocessing complete. Preprocessed images saved to {preprocessed_folder}")


In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 23.1 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import easyocr
from tqdm import tqdm

ocr_reader = easyocr.Reader(['en'], gpu=True)

input_csv = 'Train9_processed.csv'
df = pd.read_csv(input_csv)

preprocessed_folder = '/content/preprocessed_images'

def extract_text_from_image(image_path):
    try:
        results = ocr_reader.readtext(image_path)
        text = " ".join([result[1] for result in results])
        return text
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

df['extracted_text'] = ""

rows_to_remove = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting text"):
    image_path = row['image_path']

    if isinstance(image_path, str):
        image_path = os.path.join(preprocessed_folder, image_path)

        if os.path.exists(image_path):
            extracted_text = extract_text_from_image(image_path)
            if extracted_text:
                df.at[idx, 'extracted_text'] = extracted_text
            else:
                rows_to_remove.append(idx)
        else:
            print(f"Image not found: {image_path}")
            rows_to_remove.append(idx)
    else:
        print(f"Invalid image path: {image_path}")
        rows_to_remove.append(idx)

df_cleaned = df.drop(rows_to_remove).reset_index(drop=True)

output_csv = 'Train9_preprocessed_with_text.csv'
df_cleaned.to_csv(output_csv, index=False)

print(f"Text extraction complete. Cleaned data saved to {output_csv}.")


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Invalid image path: nan


Extracting text:  88%|████████▊ | 4409/4998 [09:07<01:27,  6.77it/s]

Invalid image path: nan


Extracting text: 100%|██████████| 4998/4998 [10:24<00:00,  8.01it/s]


Text extraction complete. Cleaned data saved to Train9_preprocessed_with_text.csv.


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

input_csv = 'Train9_preprocessed_with_text.csv'
df = pd.read_csv(input_csv)

group_id_encoder = LabelEncoder()
entity_name_encoder = LabelEncoder()

df['group_id_encoded'] = group_id_encoder.fit_transform(df['group_id'])
df['entity_name_encoded'] = entity_name_encoder.fit_transform(df['entity_name'])

if df['group_id_encoded'].min() == 0:
    df['group_id_encoded'] += 1

if df['entity_name_encoded'].min() == 0:
    df['entity_name_encoded'] += 1

output_csv = 'Train9_encoded.csv'
df.to_csv(output_csv, index=False)

print(f"Label encoding complete. Encoded CSV saved to {output_csv}.")


Label encoding complete. Encoded CSV saved to Train9_encoded.csv.


In [ ]:
import pandas as pd
import re
from tqdm import tqdm

input_csv = 'Train9_encoded.csv'
df = pd.read_csv(input_csv)

def preprocess_text(text):
    if not isinstance(text, str):
        return ''
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

tqdm.pandas(desc="Cleaning text")
df['cleaned_text'] = df['extracted_text'].progress_apply(preprocess_text)

output_csv = 'Train9_Text_Cleaned.csv'
df.to_csv(output_csv, index=False)

print(f"Text preprocessing complete. Updated CSV saved to {output_csv}.")


Cleaning text: 100%|██████████| 4595/4595 [00:00<00:00, 69815.42it/s]

Text preprocessing complete. Updated CSV saved to Train9_Text_Cleaned.csv.


In [ ]:
import pandas as pd
import re
from tqdm import tqdm

input_csv = 'Train9_Text_Cleaned.csv'
df = pd.read_csv(input_csv)

entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon',
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

pattern = r'(\d+(\.\d+)?)\s?(' + '|'.join(re.escape(unit) for unit in allowed_units) + r')'

def extract_numeric_unit_pairs(text):
    matches = re.findall(pattern, text, re.IGNORECASE)
    numeric_unit_pairs = [(float(num), unit.lower()) for num, _, unit in matches]
    return numeric_unit_pairs

def map_unit_to_entity(unit):
    for entity, units in entity_unit_map.items():
        if unit in units:
            return entity
    return None

def select_best_numeric_unit(text, numeric_unit_pairs):
    entity_scores = {
        'width': 0,
        'depth': 0,
        'height': 0,
        'item_weight': 0,
        'maximum_weight_recommendation': 0,
        'voltage': 0,
        'wattage': 0,
        'item_volume': 0
    }

    if 'width' in text.lower():
        entity_scores['width'] += 1
    if 'depth' in text.lower():
        entity_scores['depth'] += 1
    if 'height' in text.lower():
        entity_scores['height'] += 1
    if 'weight' in text.lower() or 'weighs' in text.lower():
        entity_scores['item_weight'] += 1
        entity_scores['maximum_weight_recommendation'] += 1
    if 'voltage' in text.lower():
        entity_scores['voltage'] += 1
    if 'watt' in text.lower():
        entity_scores['wattage'] += 1
    if 'volume' in text.lower():
        entity_scores['item_volume'] += 1

    best_pair = None
    best_score = -1

    for numeric_value, unit in numeric_unit_pairs:
        entity = map_unit_to_entity(unit)
        if entity:
            score = entity_scores.get(entity, 0)
            if score > best_score:
                best_pair = (numeric_value, unit, entity)
                best_score = score

    return best_pair

df['numeric_value'] = None
df['unit'] = None
df['entity'] = None

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting numeric values and units"):
    cleaned_text = row['cleaned_text']

    if isinstance(cleaned_text, str):
        numeric_unit_pairs = extract_numeric_unit_pairs(cleaned_text)

        if numeric_unit_pairs:
            best_pair = select_best_numeric_unit(cleaned_text, numeric_unit_pairs)

            if best_pair:
                numeric_value, unit, entity = best_pair

                df.at[idx, 'numeric_value'] = numeric_value
                df.at[idx, 'unit'] = unit
                df.at[idx, 'entity'] = entity
    else:
        print(f"Invalid cleaned_text at index {idx}: {cleaned_text}")

output_csv = 'Train9_Extracted_Best.csv'
df.to_csv(output_csv, index=False)

print(f"Numeric and unit extraction complete. Updated CSV saved to {output_csv}.")


Extracting numeric values and units:  68%|██████▊   | 3131/4595 [00:00<00:00, 15759.00it/s]

Invalid cleaned_text at index 58: nan
Invalid cleaned_text at index 3038: nan


Extracting numeric values and units: 100%|██████████| 4595/4595 [00:00<00:00, 15216.53it/s]

Invalid cleaned_text at index 3707: nan
Invalid cleaned_text at index 3710: nan
Invalid cleaned_text at index 4323: nan
Invalid cleaned_text at index 4584: nan
Invalid cleaned_text at index 4586: nan
Numeric and unit extraction complete. Updated CSV saved to Train9_Extracted_Best.csv.


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_resnet_features(image_path):
    try:
        img = image.load_img(image_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        features = resnet_model.predict(img_array)
        return features.flatten()
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return np.zeros(resnet_model.output_shape[1])

df = pd.read_csv('Train9_Extracted_Best.csv')
image_folder = '/content/downloaded_images'

features_list = []
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting ResNet50 Features"):
    image_path = os.path.join(image_folder, row['image_path'])
    features = extract_resnet_features(image_path)
    features_list.append(features)

resnet_features_df = pd.DataFrame(features_list, columns=[f'resnet_{i}' for i in range(features_list[0].shape[0])])

df_final = pd.concat([df, resnet_features_df], axis=1)

df_final.to_csv('Train9_resnet.csv', index=False)

print("ResNet50 features extracted and saved to 'Train10_resnet.csv'.")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Extracting ResNet50 Features:   0%|          | 0/4595 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step


Extracting ResNet50 Features:   0%|          | 1/4595 [00:10<12:55:13, 10.12s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


Extracting ResNet50 Features: 100%|█████████▉| 4594/4595 [08:53<00:00,  8.04it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


Extracting ResNet50 Features: 100%|██████████| 4595/4595 [08:53<00:00,  8.61it/s]


ResNet50 features extracted and saved to 'Train10_resnet.csv'.


In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from tqdm import tqdm

resnet_df = pd.read_csv('Train9_resnet.csv')

features = resnet_df.select_dtypes(include=[float, int])

imputer = SimpleImputer(strategy='mean')
features_imputed = imputer.fit_transform(features)

pca = PCA(n_components=50)

with tqdm(total=2, desc="Applying PCA") as pbar:
    pca.fit(features_imputed)
    pbar.update(1)
    reduced_features = pca.transform(features_imputed)
    pbar.update(1)

reduced_df = pd.DataFrame(reduced_features, columns=[f'pc_{i}' for i in range(1, 51)])

reduced_df.to_csv('Train9_reduced.csv', index=False)

print("Feature reduction complete. Reduced features saved to 'restnet_reduced_features.csv'.")


<ipython-input-12-1d9d556de49f>:7: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  resnet_df = pd.read_csv('Train9_resnet.csv')
Applying PCA: 100%|██████████| 2/2 [00:02<00:00,  1.21s/it]


Feature reduction complete. Reduced features saved to 'restnet_reduced_features.csv'.


In [ ]:
import pandas as pd

df_reduced = pd.read_csv('Train9_reduced.csv')
df_cleaned = pd.read_csv('Train9_Extracted_Best.csv')

print("Columns in df_reduced:", df_reduced.columns)
print("Columns in df_cleaned:", df_cleaned.columns)

if 'Unnamed: 0' in df_cleaned.columns:
    df_cleaned.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    df_cleaned.set_index('index', inplace=True)
else:
    df_cleaned.reset_index(drop=True, inplace=True)
    df_cleaned['index'] = df_cleaned.index
    df_cleaned.set_index('index', inplace=True)

if 'Unnamed: 0' in df_reduced.columns:
    df_reduced.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    df_reduced.set_index('index', inplace=True)
else:
    df_reduced.reset_index(drop=True, inplace=True)
    df_reduced['index'] = df_reduced.index
    df_reduced.set_index('index', inplace=True)

df_combined = df_cleaned.join(df_reduced, how='inner', rsuffix='_reduced')

df_combined.sort_index(inplace=True)

df_combined.reset_index(inplace=True)

df_combined.to_csv('Train9_combined_features.csv', index=False)

print("Features from 'train10_reduced_features.csv' and 'train10_cleaned.csv' combined and saved to 'train10_combined_features.csv'.")


Columns in df_reduced: Index(['pc_1', 'pc_2', 'pc_3', 'pc_4', 'pc_5', 'pc_6', 'pc_7', 'pc_8', 'pc_9',
       'pc_10', 'pc_11', 'pc_12', 'pc_13', 'pc_14', 'pc_15', 'pc_16', 'pc_17',
       'pc_18', 'pc_19', 'pc_20', 'pc_21', 'pc_22', 'pc_23', 'pc_24', 'pc_25',
       'pc_26', 'pc_27', 'pc_28', 'pc_29', 'pc_30', 'pc_31', 'pc_32', 'pc_33',
       'pc_34', 'pc_35', 'pc_36', 'pc_37', 'pc_38', 'pc_39', 'pc_40', 'pc_41',
       'pc_42', 'pc_43', 'pc_44', 'pc_45', 'pc_46', 'pc_47', 'pc_48', 'pc_49',
       'pc_50'],
      dtype='object')
Columns in df_cleaned: Index(['Unnamed: 0', 'image_link', 'group_id', 'entity_name', 'entity_value',
       'image_path', 'extracted_text', 'group_id_encoded',
       'entity_name_encoded', 'cleaned_text', 'numeric_value', 'unit',
       'entity'],
      dtype='object')
Features from 'train10_reduced_features.csv' and 'train10_cleaned.csv' combined and saved to 'train10_combined_features.csv'.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import re

df_combined = pd.read_csv('Train9_combined_features.csv')

df_combined = df_combined.drop(columns=['image_link', 'image_path'], errors='ignore')

def extract_value_and_unit(entity_value):
    match = re.match(r"([0-9.]+)\s*([a-zA-Z\s]+)", entity_value)
    if match:
        value = float(match.group(1))
        unit = match.group(2).strip()
        return value, unit
    return None, ''

df_combined[['numeric_value', 'unit']] = df_combined['entity_value'].apply(extract_value_and_unit).apply(pd.Series)

df_combined = df_combined[df_combined['numeric_value'].notna()]

df_combined = df_combined.drop(columns=['entity_value'])

X = df_combined.drop(columns=['index', 'numeric_value', 'unit'])
y_value = df_combined['numeric_value']
y_unit = df_combined['unit']

X = X.apply(pd.to_numeric, errors='ignore')
X = X.select_dtypes(include=['number'])

unit_encoder = LabelEncoder()
y_unit_encoded = unit_encoder.fit_transform(y_unit)

X_train, X_test, y_value_train, y_value_test, y_unit_train, y_unit_test = train_test_split(
    X, y_value, y_unit_encoded, test_size=0.2, random_state=42
)

tqdm.write("Training numeric value model...")
value_model = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
value_model.fit(X_train, y_value_train)

y_value_pred = value_model.predict(X_test)
mse = mean_squared_error(y_value_test, y_value_pred)
tqdm.write(f"Mean Squared Error for numeric value prediction: {mse}")

tqdm.write("Training unit prediction model...")
unit_model = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
unit_model.fit(X_train, y_unit_train)

y_unit_pred = unit_model.predict(X_test)
accuracy = accuracy_score(y_unit_test, y_unit_pred)
tqdm.write(f"Accuracy for unit prediction: {accuracy}")

test_data = X_test.copy()
test_data['numeric_value'] = y_value_test
test_data['unit'] = y_unit_test
test_data['index'] = df_combined.loc[X_test.index, 'index']
test_data.to_csv('test9.csv', index=False)

test_data = pd.read_csv('test_combined_features.csv')
test_X = test_data.drop(columns=['numeric_value', 'index'])

test_X = test_X[X.columns]

test_X = test_X.apply(pd.to_numeric, errors='ignore')
test_X = test_X.select_dtypes(include=['number'])

test_value_predictions = value_model.predict(test_X)
test_unit_predictions = unit_model.predict(test_X)

test_unit_predictions = unit_encoder.inverse_transform(test_unit_predictions)

def format_value(value):
    if value.is_integer():
        return int(value)
    else:
        return round(value, 2)

test_value_predictions = [format_value(value) for value in test_value_predictions]

test_data['entity_value'] = [f"{value} {unit}" for value, unit in zip(test_value_predictions, test_unit_predictions)]

test_data['entity_value'] = test_data['entity_value'].replace('', '*')

test_data[['index', 'entity_value']].sort_values(by='index').to_csv('output0_5.csv', index=False)
tqdm.write("Predictions saved to 'output0_5.csv'.")


Training numeric value model...
Mean Squared Error for numeric value prediction: 2.2583250625187916e+22
Training unit prediction model...
Accuracy for unit prediction: 0.637910085054678
Predictions saved to 'output0_5.csv'.


In [ ]:
import joblib

joblib.dump(value_model, 'value_model9.pkl')

joblib.dump(unit_model, 'unit_model9.pkl')

joblib.dump(unit_encoder, 'unit_encoder9.pkl')


['unit_encoder9.pkl']

In [ ]:
import pandas as pd
import joblib
import re

value_model = joblib.load('value_model9.pkl')
unit_model = joblib.load('unit_model9.pkl')
unit_encoder = joblib.load('unit_encoder9.pkl')

test_data = pd.read_csv('test_combined_features.csv')

def extract_value_and_unit(entity_name):
    match = re.match(r"([0-9.]+)\s*([a-zA-Z\s]+)", entity_name)
    if match:
        value = float(match.group(1))
        unit = match.group(2).strip()
        return value, unit
    return None, ''

test_data[['numeric_value', 'unit']] = test_data['entity_name'].apply(extract_value_and_unit).apply(pd.Series)

test_data = test_data[test_data['numeric_value'].notna()]

test_data = test_data.drop(columns=['entity_name'])

train_data = pd.read_csv('Train9_combined_features.csv')
X_train = train_data.drop(columns=['index', 'numeric_value', 'unit'])
feature_columns = X_train.columns

test_features = test_data.copy()

for col in feature_columns:
    if col not in test_features.columns:
        test_features[col] = 0

test_features = test_features[feature_columns]

test_features = test_features.apply(pd.to_numeric, errors='ignore')

test_value_predictions = value_model.predict(test_features)

test_unit_predictions = unit_model.predict(test_features)

test_unit_predictions = unit_encoder.inverse_transform(test_unit_predictions)

def format_value(value):
    if value.is_integer():
        return int(value)
    else:
        return round(value, 2)

test_value_predictions = [format_value(value) for value in test_value_predictions]

test_data['entity_value'] = [f"{value} {unit}" for value, unit in zip(test_value_predictions, test_unit_predictions)]

output_data = test_data[['index', 'entity_value']].sort_values(by='index')

output_csv = 'output0_5.csv'
output_data.to_csv(output_csv, index=False)

print(f"Predictions saved to {output_csv}")
